In [0]:
df = spark.table("default.tweets_20250623_001833")
df.printSchema()
df.show(truncate=False)

In [0]:
%pip install textblob

In [0]:
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# Define sentiment analysis function
def get_sentiment(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return 0.0

# Register UDF
sentiment_udf = udf(get_sentiment, FloatType())

# Add sentiment score column
df_sentiment = df.withColumn("sentiment", sentiment_udf(df["text"]))
df_sentiment.select("text", "sentiment").show(truncate=False)

In [0]:
df_sentiment.write.mode("overwrite").saveAsTable("default.tweets_sentiment_gold")